# Sampling Exactly from a CRF

_Posted: 01/17/2020_

Welcome to my first ever blog-post. I'm not really sure what I'm generally going to blog about, but I feel like this is a better outlet for my thoughts than just writing into private notes and google docs. And maybe this post will even help someone! 

This post is about a problem that I ran into sometime around the beginning of the fall semester, while I was working on my [ACL 2020 submission](https://damueller.com/papers/pdfs/mueller-et-al-sources-of-transfer.pdf).

The essence of the problem was that my empirical Fisher information matrices weren't salient enough for what I needed; They just seemed really noisy.
After some googling, I stumbled upon [Ferenc's blog post](https://www.inference.vc/on-empirical-fisher-information/), describing  issues with empirical fisher information matrices, mainly stemming from estimating the expectation $\mathbb{E}_{y \sim p(\mathcal{Y} | x^{(i)})}$ with just the single gold label $y^{(i)}$ from data.

In the task I was interested in, Named Entity Recognition, the output space $\mathcal{Y}$ is often intractably large, so I couldn't compute this expectation directly. However, a simple way to improve my _estimate_ of the expecation is to sample multiple times from the distribution $p_\theta(\mathcal{Y} | x^{(i)})$. In my case, this distribution was defined by a CRF, whose input logits came from a Deep BiLSTM.

A simple way to sample from this distribution would be to use approximate Markov-Chain Monte-Carlo  methods, such as [Gibb's sampling](https://en.wikipedia.org/wiki/Gibbs_sampling#:~:text=In%20statistics%2C%20Gibbs%20sampling%20or,when%20direct%20sampling%20is%20difficult.). However, these approximate methods have other hyper-parameters that I didn't really want to spend time messing around with, such as how long they take to mix. Instead, it seemed intuitive that I _should_ be able to just sample directly from the distribution $p_\theta(\mathcal{Y} | x^{(i)})$ using some kind of dynamic programming algorithm. After all, we have dynamic programs for doing other operations over a CRF. However, after doing quite a substantial amount of searching, I couldn't really find _anything_ about how to sample exactly from a CRF or HMM in an efficient way.

At this point, you might be shaking your head saying the algorithm to sample with is totally obvious, and that's why nobody mentions it. I want to agree with you... after working it out, it is extremely intuitive. There's nothing special or tricky about sampling exactly from a CRF, compared to, say, maximization with the Viterbi algorithm.
Nevertheless, it took me basically a whole working day to work out and verify, formally.
So this post is for anyone who is similarly googling for any bit of information around sampling from a CRF distribution.
I hope this is a more useful starting point than what I found.

\*Note that any of what I discuss below can also be applied to an HMM by simply replacing all scoring functions with prior probaiblities.

## An example CRF and some background

Let's consider a small, example CRF. This CRF has two labels, H and C, at each timestep $t$, over $3$ timesteps. The lattice of this CRF, desribing all possible output paths, looks like:

![](blog_figs/sample_crf.png)

The _score_ of a given sequence, or path, is found by multiplying the score of all emission scores, $\Phi_E$, and transition scores, $\Phi_T$, along the sequence. For example, the sequence $y = HCH$ looks like:

![](blog_figs/sample_crf_marked.png)

and it's score is the product of all $\Phi$'s which are red: $0.3*0.4*0.5*0.3*.4 = 0.0072$.

In this CRF, there are $2^3$ possible output sequences in this lattice. We'll label this set of sequences as $\mathcal{Y}$. Each sequence $y \in \mathcal{Y}$ has a score of:

| Path | Score |
|:---:|---:|
|$y =HHH$ |0.0216|
|$y =HHC$ |0.0216 |
|$y =HCH$ |0.0072 |
|$y =HCC$ |0.0252 |
|$y =CCC$|0.1029 |
|$y =CCH$|0.0294 |
|$y =CHC$|0.0252 |
|$y =CHH$|0.0252 |


If we sum up the scores of all sequences we get $Z = 0.2583$, which is called the partition function.
We can divide the score of each sequence by $Z$ to get the global probability distribution $P(\mathcal{Y})$ (normally it's $P(\mathcal{Y}|x)$, but I'm going to ignore the $x$ for now, and assume it's baked into our emission scores):

| Path | Probability |
|:---:|---:|
|$P(y = HHH)$ |8.36%|
|$P(y =HHC)$ | 8.36%|
|$P(y =HCH)$ | 2.79%|
|$P(y =HCC)$ | 9.75%|
|$P(y =CCC)$|39.83%|
|$P(y =CCH)$|11.38%|
|$P(y =CHC)$|9.75%|
|$P(y =CHH)$|9.75%|



## Sampling Sequences

If I want to sample a sequence from this distribution, I can just randomly choose from my 8 options in proportion to their probabilities, i.e. I should randomly sample the sequence $y = HHH$ $8.3\%$ of the time.

This is fine if I have 8 total sequences. What if my sequence was 20 words long? Then I need to enumerate $2^{20} = 1,048,576$ sequences in order to sample like this. What if I have $7$ possible labels instead of $2$? Well, then I need to enumerate $7^{20} = 7.9792266e16$ sequences. This is close to the average numbers for the specific task I was interested in, although the sequence lengths _could_ sometimes get over $100$. 

Luckily we have a nice dynamic program, called the [forward-backward algorithm](https://en.wikipedia.org/wiki/Forward%E2%80%93backward_algorithm), for computing the partition function, $Z$, and giving us token-level distributions $P(y_t = C)$. This algorithm runs in only $\mathcal{O}(TN^2)$, where $T$ is the length of the sequence, and $N$ is the number of output tags. We also have the [Viterbi algorithm](https://en.wikipedia.org/wiki/Viterbi_algorithm) for finding the top-scoring sequence which runs in similar time. These are important for both training and prediction, which is what we are usually concerned with.

But what about for sampling entire sequences? Is there an efficient way to sample from the global probability distribution over all sequences $y \sim P(\mathcal{Y})$?


### Sampling with the Posterior

Recall that the Forward-Backward algorithm gave us posterior probabilities of the form $P(y_t = C)$, i.e. token level probabilities.
We could sample an output at each timestep based on these distributions. This isn't a terrible idea... after all, we use these distributions to compute our gradients during training. But we wouldn't be sampling _sequences_ from the global distribution over _sequences_. 

To see why this is problematic, imagine that there were only two possible sequences: $y = HHH$ and $y = CCC$, each with probability $P(y) = \frac{1}{2}$. The posteriors for each timestep are also $P(y_t = H) = \frac{1}{2}$ and $P(y_t = C) = \frac{1}{2}$. If we sample a sequence from these distributions, then at each timestep we have an equal chance of sampling an $H$ or a $C$. But that means that we have a $\frac{1}{8}$ probability of sampling the sequence $y = HCH$, **_even though this sequence has probability 0 under the global sequence distribution_**.

So we need something else.

### Sampling backwards

In fact, there is a nice linear method to sample from global distribution of sequences _after_ we have the $\alpha$ scores from the forward pass of the Forward-Backward algorithm.
The $\alpha$ score of a node in a lattice represents the cumulative score of all possible paths that _end_ at that node. For example, the $\alpha$ at time-step 2 for the $H$-node is the sum of the scores of the partial paths $HH$ and $CH$.

Here is our example CRF from earlier with the $\alpha$ scores:

![](blog_figs/sample_crf_alphas.png)

Notice that if we sum the $\alpha$ scores at time step $t=3$, we get $0.0834+0.1749 = 0.2583$, which is our partition function, $Z$! This is important - we can sample the final output, $y_3$ in proportion to the global probability that all sequences end in $y_3$. For example, here the probabilities are 
$$
P(y_3 = H) = 32.29\%
$$

$$
P(y_3 = C) = 67.71\%
$$

If we sum up the probabilities of $y=HHH$, $y=HCH$, $y=CCH$, and $y=CHH$ we get $32.29\%$. Similarly for all sequences which end in $y_3 = C$, we get $67.71\%$. 

Now, imagine that we sample $y_3 = H$ with probability $32.29\%$. The question now is whether we can sample one more step backwards, while  staying consistent with the probability distributon over all sequences. This can be done, which is the whole point of this post. The way to accomplish this is by sampling the node at time step $t = 2$ in proportion to it's $\alpha_2$ score times it's score of transitioning the $y_3$.

The quasi proof can be seen with arithmetic. If we have sampled $y_3 = H$, then the score + transition scores of each output at timestep $t=2$ is $P(y_2 = H | y_3 = H) \propto \alpha_{2, H} * \Phi_T(H,H)$ and $P(y_2 = C | y_3 = H) \propto \alpha_{2, C} * \Phi_T(C,H)$. The actual values of these probabilities are:

$$
P(y_2 = H | y_3 = H) = \frac{0.195*0.6}{0.2085} = 56.11\%
$$

$$
P(y_2 = C | y_3 = H) = \frac{0.305*0.3}{0.2085} = 43.89\%
$$

Are these probabilities consistent with our global distribution? The probability of backwards sampling $HH$ is equal to 
$$
P(y_3 = H) * P(y_2 = H | y_3 = H) = 56.11\% * 32.29\% = 18.11\%
$$
There are 2 sequences which end in $HH$: $P(y = HHH) = 8.36\%$ and $P(y = CHH) = 9.75\%$, which combined equals $18.11\%$. So indeed, the probability that we sample a sequence that ends in $HH$ is equal to the global distribution over all sequences that end in $HH$. This holds for sequences which end in $CH$ as well, and in fact generally holds for all sequences!

### General Proof

The more general proof is as follows:
    
For a CRF of length $T$ and $|\mathcal{N}| = N$ labels, an $\alpha_{i, n}$ for $i = 2, ..., T$ and $n \in \mathcal{N}$ is equal to:

(1) $\alpha_{i,n} = \bigg( \sum_{n' \in \mathcal{N}} \alpha_{i-1, n'}*\Phi_T(n', n) \bigg) \Phi_E^{(t)}(n)$

(2) $\alpha_{1, n} = \Phi_E^{(1)}(n)$.

Then, the probability of sampling any sequence $\hat y = \{y_i, y_{i+1} \ldots y_T\}$ is:

$$
P(\hat y) = \frac{\alpha_{T,y_T}}{\sum_{n' \in \mathcal{N}} \alpha_{T, n'}} * \ldots * \frac{\alpha_{i+1,y_{i+1}}*\Phi_T(y_{i+2}, y_{i+1})}{\sum_{n' \in \mathcal{N}} \alpha_{i, n'} * \Phi_T(y_{i+2}, n')} * \frac{\alpha_{i,y_i} * \Phi_T(y_{i+1}, y_i)}{\sum_{n' \in \mathcal{N}} \alpha_{i, n'} * \Phi_T(y_{i+1}, n')}
$$

By (1) we know that $\frac{\alpha_{i,n}}{ \sum_{n' \in N} \alpha_{i-1, n'} \Phi_T(n', n)} = \Phi_E^{(i)}(n)$. Therefore:

$$
P(\hat y) = \frac{\Phi_E{T,y_T} * \ldots * \Phi_E(y_{i+1}) * \Phi_T(y_{i+2},y_{i+1}) * \alpha_{i, y_i} * \Phi_T(y_{i+1},y_{i})}{\sum_{n' \in \mathcal{N}} \alpha_{T, n'_T}} 
$$

Finally, recall that $\sum_{n' \in \mathcal{N}} \alpha_{T, n'_T} = Z$, the partition function.
Thus, $P(\hat y)$ is exactly equal to the _total_ probability of all sequences $y$ which end in $y'$.

It follows that for a complete sequence, $y' = \{y_1, y_2, \ldots, y_T\}$, the probability of sampling the sequence is equal to:

$$
P(y') = \frac{\Phi_E^{(T)}(y_T)  * \ldots * \Phi_E^{(2)}(y_{2}) * \Phi_T(y_{3},y_{2}) * \alpha_{1, y_1} * \Phi_T(y_2, y_1)}{Z}
$$

which, by (2) implies:

$$
P(y') = \frac{\Phi_E^{(T)}(y_T)  * \ldots * \Phi_E^{(2)}(y_2) * \Phi_T(y_{3},y_{2}) * \Phi_E^{(1)}{1, y_1}  * \Phi_T(y_2, y_1)}{Z}
$$

which is exactly the globally normalized probability of a sequence $y'$.

So, this algorithm will exactly sample from the global probability distribution, $P(\mathcal{Y})$, in time $\mathcal{O}(NT)$. However, computing the $\alpha$ scores takes $\mathcal{O}(TN^2)$ time, so that's really the upper bound on this.

Again, this may seem very intuitive to you, or extremely obvious. I can't really argue with that. However, when I was trying to figure out how to do it, I couldn't find _any_ resources about how to sample _sequences_ exactly from a CRF or HMM (Note that if you swap out the score functions for prior probabilities, this can work for an HMM in the same way).
So I figured I would share this, just in case you are searching for something similar and can't find any starting points.

## Psuedo-code

Here's an example implementation in python - I'm assuming that the alphas have already been computed and are being passed into the function along with the CRF.

In [2]:
def backward_sample(model: crf.CRF, alphas):
    choices = []
   
    initial_choice = sample(alphas[-1])
    choices.append(initial_choice)
    for i in range(model.seq_len - 2, -1, -1):
        prev_alphas = alphas[i] 
        curr_choice = choices[-1]
        tr_scores = np.array([model.transition_score(j, curr_choice) \
                              for j in range(model.n_state)])
        weights = prev_alphas + tr_scores
        next_choice = sample(weights)
        choices.append(next_choice)

    choices.reverse()
    return choices

def sample(weights):
    return np.random.choice([i for i in range(len(weights))],
                      size=None, # single choice
                      p=softmax(weights))


NameError: name 'crf' is not defined

This is the code I used to perform the sampling in my paper. It's not really optimized, and it only operates on one example at a time, but it was sufficient for my purposes. The full codebase can also be found [here](https://github.com/davidandym/crf-sampling).

Thanks for reading! I hope this helps somebody out there, in the large world of people who still use graphical models for things.